# Introduction

In the epidemiological forecasting challenges that have run over the last 5 to 10 years, specific scores such as the log score and weighted interval score have been used for evaluating forecast skill. There is some mathematical theory to support the use of these scores. Specifically, they are proper scoring rules, meaning that if data are generated from a particular process, in expectation the forecasts corresponding to the data generating process will optimize the score. However, it is not clear how well these scores measure aspects of the forecasts that epidemiologists care about, or whether they might encourage or reward some undesired behavior in forecasts. The goal of this project is to develop better understanding of the implications of using these scoring rules to evaluate forecasts, and to develop or explore alternatives.

# Decision Theoretic Set Up

Currently, infectious disease forecasting exercises separate the evaluation of forecasts from the ways in which the forecasts may actually be put to use. Here, we more directly frame forecast scoring in a decision theoretic set up that captures the potential costs of decisions that are made using forecasts as inputs. Importantly, different forecast errors (such as predicting no growth when in fact exponential growth occurs, or predicting continued exponential growth when in fact trends in incidence reverse) may be more costly for some purposes than others. We begin by giving a general set up, and then hypothesize some possible cost functions that may be relevant to public health decision makers.

To make the ideas more concrete, we introduce a running example of an agency planning the distribution of a limited set of supplies across multiple geographic units. For example, we may have a limited supply of personal protective equipment or oxygen that needs to be distributed across multiple hospitals in a state. For the purpose of this article, we assume that the agency's goal is to obtain a distribution of these limited resources that minimizes an expected loss; for example, we may wish to minimize the expected number of hospital patients in need of oxygen but without access to it. This is referred to as the *Bayes action*. We note that there are other possible strategies, such as minimizing the worst-case outcome for number of patients without access to oxygen, i.e., the *minimax action*.

We index the locations under consideration by $l = 1, \ldots, L$. The vector $Y_t$ records the outcome of interest at time $t$ across all locations; for instance, this may be the number of severe hospitalizations that occur at time $t$ in each hospital system. Let $\mathcal{Y}$ denote the space of possible values for $Y_t$, and $y$ a particular element of $\mathcal{Y}$. Similarly, let $\mathcal{A}$ denote the space of possible actions, and $a$ an element of $\mathcal{A}$. For example, $a$ may represent a particular allocation of PPE or oxygen to different hospitals represented by the supply of oxygen that is supplied to each hospital. If only $M$ resources are available, the action space is the set $\mathcal{A} = \{ (a_1, \ldots, a_L): a_l \geq 0, \sum_{l=1}^L a_l = M \}$. The loss incurred by taking action $a$ when outcome $y$ occurs is denoted by $L(y, a)$. Finally, let $\mathcal{P}$ be a convex set of probability measures on $\mathcal{Y}$, and $a_P \in \mathcal{A}$ be the *Bayes action* for a measure $P \in \mathcal{P}$:
$$a_P = \text{argmin}_{a \in \mathcal{A}} \mathbb{E}_P[L(y, a)] = \text{argmin}_{a \in \mathcal{A}} \int L(y, a) \, d P(y).$$

We posit here a specific example of a loss function that may reasonably describe the setting of distributing a limited supply of resources to each of $L$ hospitals:
$$
L(y, a) = c \sum_{l = 1}^L \max(0, y_l - a_l)
$$
In other words, if $y_l > a_l$, we incur a loss proportional to the difference between the disease burden in location $l$ and the supply of resources that were directed to location $l$. On the other hand, there is no direct loss incurred if more resources than necessary are directed to location $l$, other than the implied reduction in resources available for other locations.

Forecasts are often evaluated using proper scoring rules. A scoring rule is a function $S: \mathcal{P} \times \mathcal{Y} \rightarrow \overline{\mathbb{R}}$ that maps a probability measure $P$ and observed outcome $y$ to a score that intuitively measures how consistent $y$ is with the distribution $P$. A proper scoring rule is a scoring rule such that if data are generated from a distribution $Q$, the expected score is minimized by $Q$: $\mathbb{E}_Q[S(Q, Y)] \leq \mathbb{E}_Q[S(P, Y)]$ for all $P \in \mathcal{P}$. Restating for concreteness, the condition is that $\int S(Q, y) \, d Q(y) \leq \int S(P, y) \, d Q(y)$ for all $P \in \mathcal{P}$.

Two commonly used scoring rules are the log score (LS) and the continuous ranked probability score (CRPS): 
$$
\begin{align*}
S_{LS}(P, y) &= -\log[p(y)] \text{, where $p$ is the density of $P$} \\
S_{CRPS}(P, y) &= \int_{-\infty}^{\infty} [F_P(z) - \mathbf{1}_{[y, \infty)}(z)]^2 \, d z \text{, where $F_P$ is the cumulative distribution function of $P$}
\end{align*}
$$
Additionally, as noted by Gneiting and Raftery and apparently discussed in more detail by Dawid (haven't read it yet), in a decision theoretic setting with a loss function $L$, the score defined by $S_{L}(P, y) = L(y, a_P)$ is a proper score. **Question: is it possible to reinterpret the log score and CRPS as coming out of this decision theoretic set up with some implied or "default" action?**

Given a set of $n$ observations, we can evaluate the forecast rule $P$ (alert: notational abuse) by calculating the mean score on the observed data: $\frac{1}{n} \sum_i S(P, y_i)$. As $n \rightarrow \infty$, this mean score approaches the expected score, which is minimized by the predictive distribution associated with the data generating process if the score is proper. As a result, asymptotically, the expected scores behave the same in expectation. However, there may still be other important differences between the scores: asymptotically, they may behave differently in terms of quantities other than the first moment; and even the first moment may behave differently with small sample sizes.

# Understanding implications of different loss/utility functions for optimal forecasts

Our basic goal is to understand the behavior of an optimal forecaster under different loss functions. Our understanding of the theory of proper scoring rules is that if a proper scoring rule is used and the data generating process is in the set of models under consideration, that process will ``beat" all other models in expectation (i.e., as the sample size of our evaluation set goes to infinity). However, what happens if (a) the data generating process is not among the candidate models [note that in practice, the best performing models are typically much simpler than the data generating process]; and (b) our sample size is limited?

Can we make any statements, either through formal derivation or simulation studies, about differences between the optimal forecasts under different [proper] scoring rules? We divide this question into two sub-parts:

1. Are there any situations in which a commonly used proper scoring rule leads to predictions that we find intuitively unappealing?
2. Do more carefully designed scoring rules that match targeted use cases result in qualitatively different optimal forecasts?

## Possible downsides of common scoring rules

Do common scoring rules reward behavior that is not desirable from the perspective of an epidemiologist who wants to use forecasts as an input to some decision? We have thought about two ways this might come about:
1. Scoring rules that equally reward performance at all time points may encourage forecasters to either predict no growth, or maybe to predict a continuation of recent trends in growth. Changes in growth rate are relatively rare; most of the time, a forecaster can do well by predicting a continuation of recent trends. But maybe what an epidemiologist cares about most is when trends are going to change.
2. Common scoring rules may encourage some kind of shrinkage that might encourage downward-biased forecasts of disease incidence in the largest locations or at the peaks in incidence.  Aaron Gerding found this [blog post](https://jmanton.wordpress.com/?s=james-stein) that briefly states the situation and concerns here.
3. Possibly related to the previous point, commonly used scoring rules may encourage forecasts with an attenuated slope?

Here are some expanded thoughts on the second point:

### Connection between log score and MSE in a simple case

Suppose $X_1, \ldots, X_T \sim \text{Normal}_p(\theta, \sigma^2 \mathbb{I})$ are i.i.d. $p$-dimensional random variables, with the mean vector $\theta$ unknown and, for now, $\sigma^2$ known. Our goal is to study estimators of the distribution of a new random variate $X_{T+1}$ drawn from this same distribution.  We will denote such a distributional estimator by $\widehat{\mathcal{D}}(X_1, \ldots, X_T)$ to emphasize that the estimator is a function of the random variables $X_1, \ldots, X_T$, or just $\widehat{\mathcal{D}}$ for brevity.  If we have an estimator $\widehat{\theta}$ of the mean vector $\theta$, a corresponding distributional estimator is given by $\widehat{\mathcal{D}} = \text{Normal}_p(\widehat{\theta}, \sigma^2 \mathbb{I})$.

Given observed values $x_1, \ldots, x_{T+1}$, we could measure the quality of the distributional estimate $\widehat{d} = \widehat{d}(x_1, \ldots, x_t)$ by the negative log sore:
$$NLS(\widehat{d}, x_{T+1}) = -\log\left[ f(x_{T+1} \vert \widehat{\theta}, \sigma^2\mathbb{I} ) \right],$$
where $f$ is the pdf of a multivariate normal distribution. In general, our goal is to select an estimator that minimizes the expected negative log score (NLS):

$$
\begin{align*}
\mathcal{R}_{NLS}(\widehat{\mathcal{D}}) &= \mathbb{E}_{X_{1:{T+1}}}\left[ NLS(\widehat{\mathcal{D}}, X_{T+1}) \right] \\
&= \int \cdots \int_{\mathcal{X}} -\log\left[ f(x_{T+1} \vert \widehat{\theta}(x_1, \ldots, x_T), \sigma^2\mathbb{I} ) \right] \, d F_\theta(x_1 \cdots x_{T+1}),
\end{align*}
$$

where subscript $X_{1:{T+1}}$ indicates that the expectation is with respect to the distribution of $X_1, \ldots, X_{T+1}$, and $F_\theta$ is the distribution of these random variables.

In the context of point estimation of the mean vector $\theta$, we could also measure the quality of the estimator $\widehat{\theta}$ via the mean squared error:
$$
\begin{align}
\mathcal{R}_{MSE}(\widehat{\theta}) &= \mathbb{E}_{X_{1:T}}\left[\sum_{j = 1}^p (\widehat{\theta}_j - \theta_j)^2 \right] \\
&= \int \cdots \int{\mathcal{X}} \sum_{j = 1}^p \left[\widehat{\theta}(x_1, \ldots, x_T)_j - \theta_j\right]^2 \, d F_\theta(x_1 \cdots x_{T}).
\end{align}
$$
Since this expression only evaluates the risk of the estimator of $\theta$ based on the first $T$ observations, the expection is only with respect to the distribution of $X_1, \ldots, X_T$.

In fact, these two risk functions are essentially equivalent:
$$
\begin{align*}
\mathcal{R}_{NLS}(\widehat{\mathcal{D}}) &= \mathbb{E}_{X_{1:{T+1}}}\left[ -\log\left\{ f(X_{T+1} \vert \widehat{\theta}(X_1, \ldots, X_T), \sigma^2\mathbb{I} ) \right\} \right] \\
&= \mathbb{E}_{X_{1:{T+1}}}\left[ - \sum_{j = 1}^p \left\{ c - \frac{1}{2 \sigma^2} (X_{T+1, j} - \widehat{\theta}_j)^2 \right\} \right] \\
&= -pc + \frac{1}{2 \sigma^2} \sum_{j = 1}^p \mathbb{E}_{X_{1:{T+1}}}\left[ (X_{T+1, j} - \widehat{\theta}_j)^2 \right] \\
&= -pc + \frac{1}{2 \sigma^2} \sum_{j = 1}^p \mathbb{E}_{X_{1:{T+1}}}\left[ \left\{(X_{T+1, j} - \theta_j) + (\theta_j - \widehat{\theta}_j)\right\}^2 \right] \\
&= -pc + \frac{1}{2 \sigma^2} \sum_{j = 1}^p \left[ \mathbb{E}_{X_{1:{T+1}}}\left\{(X_{T+1, j} - \theta_j)^2\right\} + 2 \mathbb{E}_{X_{1:{T+1}}}\left\{(X_{T+1, j} - \theta_j)(\theta_j - \widehat{\theta}_j)\right\} + \mathbb{E}_{X_{1:{T+1}}}\left\{(\theta_j - \widehat{\theta}_j)^2\right\} \right] \\
&= -pc + \frac{1}{2 \sigma^2} \left[ p\sigma^2 + \mathcal{R}_{MSE}(\widehat{\theta}) \right]
\end{align*}
$$

### Connection to James-Stein and MSE Optimality

In the simple setting above, the James-Stein estimator and similar shrinkage estimators are known to achieve smaller MSE than the maximum likelihood estimator.  Denote such a shrinkage estimator of $\theta$ and the corresponding estimator of the distribution of $X_{T+1}$ by $\widehat{\theta}^{S}$ and $\widehat{\mathcal{D}}^{S}$ respectively, and the similar estimators obtained from maximum likelihood by $\widehat{\theta}^{MLE}$ and $\widehat{\mathcal{D}}^{MLE}$.  The James-Stein result is that $\mathcal{R}_{MSE}(\widehat{\theta}^S) \leq \mathcal{R}_{MSE}(\widehat{\theta}^{MLE})$, with a strict inequality for almost all values of $\theta$. Combining this result with the equivalence between negative log score and MSE in Equation~\eqref{eqn:risk_equivalence} above, we see that also $\mathcal{R}_{NLS}(\widehat{\mathcal{D}}^{S}) \leq \mathcal{R}_{NLS}(\widehat{\mathcal{D}}^{MLE})$.

## Simulation Study - One Time Point

We consider a first very simple simulation study based on the conditions described above, with a single fixed time point and multiple locations:

$$
\begin{align*}
Z_t &\sim \text{Normal}_L(\theta, \sigma^2 \mathbb{I}) \\
Y_{lt} &= Z_{lt} \cdot \text{pop}_l
\end{align*}
$$

We fix $L = 50$ and choose $\theta$ to be distributed between 10 and 60; these are plausible values for hospitalizations per 100,000 population. We assume $\sigma$ is known to be 1. The number of hospitalized people in location $l$, $Y_{lt}$, is calculated as the rate of hospitalizations per 100,000 population, $Z_{lt}$, times the population in location $l$ in units of 100,000 people.

We measure distributional forecast skill through the three proper scores outlined above: $S_{LS}$, $S_{CRPS}$, and $S_L$ using the application-specific loss based on unmet hospitalization needs.

We consider two estimators of $\theta$, based on a single observation of length 50 from the target distribution:
1. the maximum likelihood estimator, which sets $\widehat{\theta} = y$
2. an empirical linear Bayes shrinkage estimator proposed by Efron and Morris

In [30]:
import numpy as np
import pandas as pd
import properscoring as ps
from scipy.stats import norm

In [22]:
location_pops = pd.read_csv("https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-locations/locations.csv")
location_pops

,abbreviation,location,location_name,population
0,US,US,US,332875137.0
1,AL,01,Alabama,4903185.0
2,AK,02,Alaska,731545.0
3,AZ,04,Arizona,7278717.0
4,AR,05,Arkansas,3017804.0
...,...,...,...,...
3197,NaN,56037,Sweetwater County,42343.0
3198,NaN,56039,Teton County,23464.0
3199,NaN,56041,Uinta County,20226.0
3200,NaN,56043,Washakie County,7805.0


In [28]:
state_pops = location_pops \
    .loc[
        (location_pops.location <= '56') &
        (location_pops.location.str.len() <= 2) &
        (location_pops.abbreviation != 'DC')] \
    .assign(pop_100k = lambda x: x.population / 100000.)
state_pops

,abbreviation,location,location_name,population,pop_100k
1,AL,01,Alabama,4903185.0,49.03185
2,AK,02,Alaska,731545.0,7.31545
3,AZ,04,Arizona,7278717.0,72.78717
4,AR,05,Arkansas,3017804.0,30.17804
5,CA,06,California,39512223.0,395.12223
6,CO,08,Colorado,5758736.0,57.58736
7,CT,09,Connecticut,3565287.0,35.65287
8,DE,10,Delaware,973764.0,9.73764
10,FL,12,Florida,21477737.0,214.77737
11,GA,13,Georgia,10617423.0,106.17423


In [39]:
theta = np.linspace(10, 60, num = 50)
supply_limit = 0.9 * np.sum(theta * state_pops.pop_100k)

def calc_theta_hat_mle(y):
    return y

def calc_theta_hat_elb(y, scale = 1.0):
    m_hat = np.mean(y)
    a_hat_plus_d_hat = np.var(y)
    a_hat = a_hat_plus_d_hat - scale**2
    c_hat = a_hat / a_hat_plus_d_hat
    return m_hat + c_hat * (y - m_hat)

nsim = 10000
results = []

for i in range(nsim):
    z_train = np.random.normal(loc = theta, scale = 1.0)
    y_train = z_train * state_pops.pop_100k
    z_test = np.random.normal(loc = theta, scale = 1.0)
    y_test = z_test * state_pops.pop_100k

    theta_hat_mle = calc_theta_hat_mle(z_train)
    theta_hat_elb = calc_theta_hat_elb(z_train)

    y_hat_mle_via_pop100k = theta_hat_mle * state_pops.pop_100k
    y_hat_elb_via_pop100k = theta_hat_elb * state_pops.pop_100k

    ls_mle_via_pop100k = np.mean(norm.logpdf(y_test, loc = y_hat_mle_via_pop100k, scale=state_pops.pop_100k, ))
    ls_elb_via_pop100k = np.mean(norm.logpdf(y_test, loc = y_hat_elb_via_pop100k, scale=state_pops.pop_100k))

    crps_mle_via_pop100k = np.mean(ps.crps_gaussian(y_test, mu = y_hat_mle_via_pop100k, sig=state_pops.pop_100k))
    crps_elb_via_pop100k = np.mean(ps.crps_gaussian(y_test, mu = y_hat_elb_via_pop100k, sig=state_pops.pop_100k))

    a_hat_mle_via_pop100k = supply_limit * (y_hat_mle_via_pop100k / np.sum(y_hat_mle_via_pop100k))
    error_mle_via_pop100k = y_test - a_hat_mle_via_pop100k
    error_mle_via_pop100k[error_mle_via_pop100k < 0] = 0
    as_mle_via_pop100k = np.sum(error_mle_via_pop100k)

    a_hat_elb_via_pop100k = supply_limit * (y_hat_elb_via_pop100k / np.sum(y_hat_elb_via_pop100k))
    error_elb_via_pop100k = y_test - a_hat_elb_via_pop100k
    error_elb_via_pop100k[error_elb_via_pop100k < 0] = 0
    as_elb_via_pop100k = np.sum(error_elb_via_pop100k)

    results.append(pd.DataFrame({
        'method': ['mle_via_pop100k', 'elb_via_pop100k'],
        'ls': [ls_mle_via_pop100k, ls_elb_via_pop100k],
        'crps': [crps_mle_via_pop100k, crps_elb_via_pop100k],
        'allocation_score': [as_mle_via_pop100k, as_elb_via_pop100k]
    }))

results = pd.concat(results)
results

,method,ls,crps,allocation_score
0,mle_via_pop100k,-5.952639,63.250287,11354.650663
1,elb_via_pop100k,-5.937826,62.270643,11357.819677
0,mle_via_pop100k,-5.586159,50.871741,10829.167770
1,elb_via_pop100k,-5.600105,50.966789,10831.474003
0,mle_via_pop100k,-5.423005,55.283052,11636.300381
...,...,...,...,...
1,elb_via_pop100k,-5.614442,63.785246,10471.513871
0,mle_via_pop100k,-5.700609,62.940052,11898.223731
1,elb_via_pop100k,-5.702380,62.056551,11910.468600
0,mle_via_pop100k,-5.873091,66.174011,10595.338010


In [40]:
results.groupby('method').mean()

,ls,crps,allocation_score
method,,,
elb_via_pop100k,-5.612103,53.640695,11102.059930
mle_via_pop100k,-5.614120,53.701211,11091.008124


According to these results, the MLE method scores worse according to log score (it has a lower log score) and CRPS (it has a higher CRPS), but better according to the application-specific allocation score (it has a lower allocation score). Our intuition is that this is because the shrinkage method helps both log score and CRPS, as indicated in the theory above, but it is harmful for the allocation score. Shrinkage is harmful to the allocation score because it tends to shrink large estimates down, resulting in systematic under-allocation of resources to the locations with largest incidence.

Follow up questions:
 * What about other estimators?
 * What about other data generating processes?

# Older commentary below.

### Reason for concern and possible next steps

The above suggests that if log scores are used for measuring forecast skill, shrinkage estimators may be rewarded. The details need to be worked out, but there are a few potential concerns here:

1. If we're measuring forecast skill across many locations, this may reward systematic underprediction of incidence in the locations with largest incidence. This would not be desirable from the perspective of an epidemiologist working in those locations.
2. If we're measuring forecast skill over time, this may reward systematic underprediction of incidence at times with largest incidence (indeed, this is exactly what a Kalman filter will tend to do). This would not be desirable from the perspective of a public health decision maker; e.g., it may be important to obtain unbiased estimates of peak hospitalizations, and this may not be what arises from forecasts of peak hospitalizations that are obtained from a model that is trained to optimize a rule that encourages shrinkage at the peaks.
3. I think the CRPS in particular may interact strangely with processes that occur with exponential growth. As used in the forecasting exercises, the CRPS measures forecast skill on the scale of observed cases or deaths. If a model estimates the growth rate of a process with exponential growth, errors on the high side will likely result in larger penalties than errors on the low side. i.e. if the growth rate is 3, an estimate of 3.1 may result in lower scores than an estimate of 2.9. This may induce a systematic downward bias in growth rate estimates. Can we compare to a score directly measuring estimates of growth rate?

There are also some follow up questions to the above:

1. What happens to the above if $\sigma^2$ is not known?
1. What happens to the above if the observations are not independent?  Suppose they arise from a linear Gaussian time series process (i.e., the set up for a Kalman filter) to start.
1. What happens if there is more structure in the covariance for the random vector observed at each time?
1. What happens if there are different variances for each component?  For example, suppose you have $p$ locations with different populations, and the mean and variance both scale with the population size?  We think it should be optimal from an MSE/log score perspective to do more shrinkage for larger locations.
1. What happens to the above if observations don't follow a normal distribution? Something skewed like a log-normal?  Something discrete like a Poisson or negative binomial?
1. Bayesian formulation of all of the above.
1. Other scoring rules (e.g. CRPS/WIS)